In [3]:
import pandas as pd
import csv

data = pd.read_csv('auctions.csv')
data

,auction_id,RoR,STR,BPL,lots,product_types,avg_start_bid,avg_reserve,auction_mech,state
0,100,1.026112,0.503097,8.324074,108,0.768519,0.000000,10568.250000,SealedBid,MD
1,101,1.407666,1.000000,5.571429,7,1.000000,0.000000,4805.617249,EnglishForward,TX
2,102,0.000000,0.000000,0.000000,2,1.000000,0.000000,3550.000000,SealedBid,MD
3,103,0.884021,1.000000,1.000000,3,0.333333,0.000000,7693.876790,FixedPrice,NE
4,104,0.972593,1.000000,4.533333,90,0.611111,0.000000,8661.111111,EnglishForward,NY
...,...,...,...,...,...,...,...,...,...,...
6215,6315,1.010807,0.280132,1.571429,115,0.796610,0.998503,15009.730254,EnglishForward,FL
6216,6316,0.997599,1.000000,1.000000,10,1.000000,0.000000,15296.658008,FixedPrice,FL
6217,6317,0.942716,0.333875,6.200000,45,0.844444,0.000000,12188.244667,EnglishForward,DE
6218,6318,1.000787,0.509448,6.758065,62,0.338710,0.000000,14172.611210,EnglishForward,NV
